In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
file = 'final2.csv'
df = pd.read_csv(file)
df = pd.DataFrame(df)
df

,Направление,Год поступления,Количество абитуриентов,Баллы ЕГЭ,Возраст,Вступительный балл,Регион рождения,Уровень безработицы,Доля женщин,Доля квоты,Доля волонтерства
0,1,1908,11,-0.397620,-0.302921,1.480258,1.480258,33.793476,0.110471,-0.932472,0.965381
1,1,1909,11,-0.323680,-1.201973,-0.165480,-0.165480,29.929197,-0.607585,0.270533,-0.579118
2,1,1910,11,-1.186287,-1.799674,-1.610080,-1.610080,27.189645,-0.639556,-0.748223,0.253945
3,1,1911,11,-0.042719,0.749308,0.567303,0.567303,32.609474,-0.455643,-0.761783,0.345629
4,1,1912,11,0.957904,0.521211,-0.084162,-0.084162,29.937970,-0.938985,-0.640584,-0.386541
...,...,...,...,...,...,...,...,...,...,...,...
5962,51,2020,19,-0.147270,0.738005,-1.956321,-1.956321,17.091670,-0.354391,0.620962,-0.424094
5963,51,2021,20,0.816437,0.244920,0.138422,0.138422,17.964293,-0.627167,-0.408057,-0.517372
5964,51,2022,20,-0.284249,0.575489,-1.744231,-1.744231,16.264727,1.417724,0.041130,-0.011007
5965,51,2023,20,-0.598730,-0.907030,-0.052447,-0.052447,18.913300,0.567138,0.897687,-0.776967


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm

X = df.drop(columns=['Количество абитуриентов'])
y = df['Количество абитуриентов']

# Создание весов классов
class_counts = y.value_counts()
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
weights = y.map(class_weights)

# Добавление весов
df['Вес'] = weights

In [52]:
# Разбиение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test, weights_train, _ = train_test_split(X, y, df['Вес'], test_size=0.2, random_state=42)
# Модель случайного леса
model_rf = RandomForestRegressor(random_state=42)
model_rf.fit(X_train, y_train, sample_weight=weights_train)
y_pred_rf = model_rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Случайный лес - MAE: {mae_rf:.2f}, RMSE: {rmse_rf:.2f}")

Случайный лес - MAE: 0.10, RMSE: 0.19


In [53]:
# Модель градиентного бустинга
model_gbr = GradientBoostingRegressor(random_state=42)
model_gbr.fit(X_train, y_train, sample_weight=weights_train)
y_pred_gbr = model_gbr.predict(X_test)
mae_gbr = mean_absolute_error(y_test, y_pred_gbr)
rmse_gbr = mean_squared_error(y_test, y_pred_gbr)
print(f"Градиентный бустинг - MAE: {mae_gbr:.2f}, RMSE: {rmse_gbr:.2f}")

Градиентный бустинг - MAE: 0.72, RMSE: 0.97


In [54]:
# Модель линейной регрессии
X_train_const = sm.add_constant(X_train)
model_sm = sm.WLS(y_train, X_train_const, weights=weights_train)
results_sm = model_sm.fit()

X_test_const = sm.add_constant(X_test)
y_pred_sm = results_sm.predict(X_test_const)

mae_sm = mean_absolute_error(y_test, y_pred_sm)
rmse_sm = mean_squared_error(y_test, y_pred_sm)
print(f"Взвешенная линейная регрессия - MAE: {mae_sm:.2f}, RMSE: {rmse_sm:.2f}")

Взвешенная линейная регрессия - MAE: 4.49, RMSE: 101.80


In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# Параметры кросс-валидации
n_iter = 10  # Количество итераций
test_size = 0.2    # Размер тестовой выборки (20%)
mse_scores = []  # Список для хранения MSE на каждой итерации 
r2_scores = []   # Список для хранения R² на каждой итерации 

for i in range(n_iter): 
    # Случайное разбиение данных на обучающую и тестовую выборки 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i) 

    # Создание и обучение модели (Линейная регрессия) 
    linear_model = LinearRegression() 
    linear_model.fit(X_train, y_train) 

    # Предсказание 
    y_pred = linear_model.predict(X_test) 

    # Оценка производительности модели 
    mse = mean_squared_error(y_test, y_pred)  # Среднеквадратичная ошибка
    r2 = r2_score(y_test, y_pred)              # Коэффициент детерминации

    mse_scores.append(mse) 
    r2_scores.append(r2)

    print(f'Итерация {i + 1}: MSE = {mse:.2f}, R² = {r2:.2f}') 

# Средние значения за все итерации 
mean_mse = np.mean(mse_scores) 
mean_r2 = np.mean(r2_scores)

print(f'Средняя MSE за {n_iter} итераций: {mean_mse:.2f}')
print(f'Средний R² за {n_iter} итераций: {mean_r2:.2f}')

Итерация 1: MSE = 22.41, R² = 0.43
Итерация 2: MSE = 24.63, R² = 0.41
Итерация 3: MSE = 24.98, R² = 0.37
Итерация 4: MSE = 20.42, R² = 0.41
Итерация 5: MSE = 30.83, R² = 0.31
Итерация 6: MSE = 20.45, R² = 0.45
Итерация 7: MSE = 37.33, R² = 0.10
Итерация 8: MSE = 22.21, R² = 0.41
Итерация 9: MSE = 23.12, R² = 0.38
Итерация 10: MSE = 21.64, R² = 0.42
Средняя MSE за 10 итераций: 24.80
Средний R² за 10 итераций: 0.37


In [23]:
from xgboost import XGBRegressor

# Параметры кросс-валидации 
n_iter = 10  # Количество итераций 
test_size = 0.2  # Размер тестовой выборки (20%) 

mse_scores = []  # Список для хранения MSE на каждой итерации 
r2_scores = []   # Список для хранения R² на каждой итерации 

for i in range(n_iter): 
    # Случайное разбиение данных на обучающую и тестовую выборки 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i) 

    # Создание и обучение модели XGBoost
    xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
    xgb_model.fit(X_train, y_train) 

    # Предсказание 
    y_pred = xgb_model.predict(X_test) 

    # Оценка производительности модели 
    mse = mean_squared_error(y_test, y_pred)  # Среднеквадратичная ошибка
    r2 = r2_score(y_test, y_pred)              # Коэффициент детерминации

    mse_scores.append(mse) 
    r2_scores.append(r2)

    print(f'Итерация {i + 1}: MSE = {mse:.2f}, R² = {r2:.2f}') 

# Средние значения за все итерации 
mean_mse = np.mean(mse_scores) 
mean_r2 = np.mean(r2_scores)

print(f'Средняя MSE за {n_iter} итераций: {mean_mse:.2f}')
print(f'Средний R² за {n_iter} итераций: {mean_r2:.2f}')

Итерация 1: MSE = 7.21, R² = 0.82
Итерация 2: MSE = 7.13, R² = 0.83
Итерация 3: MSE = 6.25, R² = 0.84
Итерация 4: MSE = 6.67, R² = 0.81
Итерация 5: MSE = 6.81, R² = 0.85
Итерация 6: MSE = 4.41, R² = 0.88
Итерация 7: MSE = 8.58, R² = 0.79
Итерация 8: MSE = 4.53, R² = 0.88
Итерация 9: MSE = 5.49, R² = 0.85
Итерация 10: MSE = 5.79, R² = 0.85
Средняя MSE за 10 итераций: 6.29
Средний R² за 10 итераций: 0.84


Задача прогнозирования - частный случай задачи регрессии.
Попробовать обучить с помощью:
1. многослойного перспетрона
2. радиально-базисной сетью
3. обобщенно-регрессионной сетью
4. нейросеть регрессии

In [ ]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Произошел 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Произошел 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Произошел 